In [40]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error, precision_score, recall_score, roc_auc_score
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [41]:
# Load dataframes
df_recipes = pd.read_csv("../data/df_with_diets.csv")
df_reviews = pd.read_csv("../data/reviews.csv")

### Adjust rating values

In [42]:
# Change rating in order to make initial given 0 count unlike the 0 given for mising ratings
def update_rating(value, updated):
    if updated:
        return value
    if value == 0:
        return 1, True
    elif value == 1:
        return 2, True
    elif value == 2:
        return 3, True
    elif value == 3:
        return 4, True
    elif value == 4:
        return 5, True
    elif value == 5:
        return 6, True
    else:
        return value, False

# Creates a new column to check if values are already adjusted
df_reviews['Updated'] = False

# # Apply the function to the entire column "Rating" in the DataFrame "df_reviews"
df_reviews['Rating'], df_reviews['Updated'] = zip(*df_reviews.apply(lambda row: update_rating(row['Rating'], row['Updated']), axis=1))

# Removing the auxiliary column 'Updated
del df_reviews['Updated']

#### Reduce dataset by min. No of reviews for author & ratings

In [43]:
# Filter only recipes with more than 10 reviews
ratings = df_reviews.copy()
recipe_group = ratings.groupby("RecipeId").filter(lambda x: x["RecipeId"].count() > 10)

In [44]:
# Filter only authors with more than 5 reviews
user_rating_group = ratings.groupby(['AuthorId']).filter(lambda x: x["AuthorId"].count() > 5)

In [45]:
# Adjust initial dataset (ratings) according prerunned filters

ratings = ratings[ratings['AuthorId'].isin(user_rating_group['AuthorId'])]
ratings = ratings[ratings['RecipeId'].isin(recipe_group['RecipeId'])]


In [46]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 544469 entries, 5 to 1401942
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ReviewId       544469 non-null  int64 
 1   RecipeId       544469 non-null  int64 
 2   AuthorId       544469 non-null  int64 
 3   AuthorName     544469 non-null  object
 4   Rating         544469 non-null  int64 
 5   Review         544465 non-null  object
 6   DateSubmitted  544469 non-null  object
 7   DateModified   544469 non-null  object
dtypes: int64(4), object(4)
memory usage: 37.4+ MB


In [47]:
# Create list with vegan recipeIds
vegan_recipes = df_recipes[df_recipes['vegan'] == 1]['RecipeId'].tolist()

# Keep only rows of RecipeIds which are element of vegan recipe list
ratings_filtered = ratings[ratings['RecipeId'].isin(vegan_recipes)]
vegan = ratings_filtered.copy()

In [48]:
# Cross validation if chicken recipe did not sneak into dataset
recipe_id = 76
is_in_vegan = recipe_id in vegan.index
print(is_in_vegan)

False


In [49]:
# Drop unrelevant columns
vegan.drop(['ReviewId', 'AuthorName', 'Review', 'DateSubmitted', 'DateModified'], axis=1, inplace=True)

In [50]:
vegan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21400 entries, 724 to 1401851
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   RecipeId  21400 non-null  int64
 1   AuthorId  21400 non-null  int64
 2   Rating    21400 non-null  int64
dtypes: int64(3)
memory usage: 1.2 MB


## Add test user

#### Add vegan & italien food lover

In [51]:
new_row = {'RecipeId': 29253, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row
print(vegan.tail(1))

       RecipeId  AuthorId  Rating
21400     29253     10700       6


In [52]:
new_row_1 = {'RecipeId': 358, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_1
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21400     29253     10700       6
21401       358     10700       6


In [53]:
new_row_2 = {'RecipeId': 35805, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_2
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21401       358     10700       6
21402     35805     10700       6


In [54]:
new_row_3 = {'RecipeId': 217734, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_3
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21402     35805     10700       6
21403    217734     10700       6


In [55]:
new_row_4 = {'RecipeId': 5413, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_4
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21403    217734     10700       6
21404      5413     10700       6


In [56]:
new_row_5 = {'RecipeId': 403282, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_5
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21404      5413     10700       6
21405    403282     10700       6


In [57]:
new_row_6 = {'RecipeId': 115553, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_6
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21405    403282     10700       6
21406    115553     10700       6


In [58]:
new_row_7 = {'RecipeId': 81419, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_7
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21406    115553     10700       6
21407     81419     10700       6


In [59]:
new_row_8 = {'RecipeId': 175799, 'AuthorId': 10700, 'Rating': 6}

vegan.loc[len(vegan)] = new_row_8
print(vegan.tail(2))

       RecipeId  AuthorId  Rating
21407     81419     10700       6
21408    175799     10700       6


In [60]:
# Cross validation if chicken recipe did not sneak into dataset
recipe_id = 76
is_in_vegan = recipe_id in vegan.index
print(is_in_vegan)

False


In [61]:
vegan.head(), vegan.shape

(      RecipeId  AuthorId  Rating
 724       8935     10678       6
 821       8970     11455       1
 822       9267     10678       5
 966       9180      6338       5
 1076      8949      6897       5,
 (21409, 3))

## Create matrix for models


In [62]:
# Create table with AuthorId as columns, RecipeId as index & ratings as value
vegan = vegan.pivot_table(index='RecipeId', columns='AuthorId', values='Rating')

# Replace NaN with 0
vegan.fillna(0, inplace=True)

In [63]:
# Make copy for NMF model
vegan_2 = vegan.copy()

## KNN

In [64]:
# Define AuthorId for which model should calculate recommendations
specific_author_id = 10700 # HERE THE AUTHORID NEEDS TO BE ADJUSTED!

# Extract Ratings for specific AuthorId
query_data = vegan.loc[:, specific_author_id].values.reshape(1, -1)

# Initialize and fit the Nearest Neighbors model
n_neighbors = min(600, vegan.shape[0]) # HERE NO OF RECOMMENDATIONS NEEDS TO BE ADJUSTED ACCORDING TO NEED (size probably does not need to be as big as for general model no needed)
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_neighbors)
knn_model.fit(vegan.T)  # Transpose the DataFrame to obtain the correct shape

# Find the indices of the nearest neighbors for the specific AuthorId
distances, neighbor_indices = knn_model.kneighbors(query_data)

# Make sure that index corresponds with expected value
valid_indices = [idx for idx in neighbor_indices.flatten() if idx < vegan.shape[0]]

# Output the RecipeIds of the nearest neighbors
nearest_recipe_ids = vegan.index[valid_indices]

# Retrieve the corresponding names for the RecipeIds
nearest_recipe_names = df_recipes[df_recipes['RecipeId'].isin(nearest_recipe_ids)]['Name'].values

# Create a DataFrame with the RecipeIds and their names
nearest_recipes_df = pd.DataFrame({'RecipeId': nearest_recipe_ids, 'Name': nearest_recipe_names})

#create dataframe for streamlit
nearest_recipe_ids_df = pd.DataFrame({'RecipeId': nearest_recipe_ids})

# Sort the DataFrame by the smallest distance
# nearest_recipes_df_sorted = nearest_recipes_df.sort_values(by='RecipeId')

# Give RecipeIds & Names
print("Recommendations for AuthorId:", specific_author_id)
print(nearest_recipes_df)

Recommendations for AuthorId: 10700
    RecipeId                                               Name
0      14306                       Spicy Hash Browns - Homemade
1      97196                                      Pumpkin Puree
2      96820                               Low Calorie Lemonade
3     133469                          Vegan Macaroni and Cheese
4     239129                     Pickled Garlic with Hot Pepper
5     313788                  Homemade Low Calorie Potato Chips
6     450153                               Moroccan Lentil Soup
7      29671                Lemon Roasted Potatoes, Greek Style
8     278278                       Cajun Style Oven Hash Browns
9      38584                                     Tofu Nut Balls
10     61725                             Freezer Strawberry Jam
11    150898                        Canary Island Cilantro Soup
12    217244                Polish Dill Pickles Made in a Crock
13     77326             Yummy and Super Easy Crock Pot Oatmeal
14  

---
Uncommend to download for streamlit:

In [65]:
# CSV of recommendations for streamlit
# nearest_recipe_ids_df.to_csv("../data/knn_vegan_veganitalian.csv")

---

### NMF


In [66]:
# Initalization of model

model = NMF(n_components=6)
model.fit(vegan_2)

NMF(n_components=6)

In [67]:
# Calculation of 2 lower dimensional matrices
H = pd.DataFrame(model.components_)
W = pd.DataFrame(model.transform(vegan_2))

# Create new df with dot product of W & H
V = pd.DataFrame(np.dot(W, H), columns=vegan_2.columns)
V.index = vegan_2.index

V.tail()

AuthorId,1533,1535,1792,2148,2310,2312,2586,2695,3205,3288,...,2001928915,2002048922,2002084712,2002169932,2002256447,2002273175,2002375343,2002404048,2002448241,2002754832
RecipeId,,,,,,,,,,,,,,,,,,,,,
484546,0.000483,0.002186,0.000120,0.000047,0.001550,0.000059,0.000691,0.000134,0.000631,0.000353,...,0.000322,0.000033,0.000029,0.000109,0.000572,0.000101,0.000555,0.000005,0.000043,0.000369
485857,0.001372,0.034870,0.000717,0.000248,0.031914,0.001026,0.000000,0.000249,0.000067,0.000943,...,0.000812,0.000104,0.000096,0.000745,0.000000,0.000464,0.001483,0.000084,0.000128,0.001237
490247,0.000815,0.014592,0.000616,0.000355,0.010310,0.000590,0.005288,0.000845,0.004826,0.000296,...,0.000698,0.000047,0.000025,0.000492,0.004375,0.000138,0.000465,0.000034,0.000042,0.000629
490362,0.000671,0.003451,0.000268,0.000128,0.001871,0.000131,0.002199,0.000353,0.002007,0.000426,...,0.000499,0.000042,0.000036,0.000187,0.001819,0.000121,0.000670,0.000010,0.000051,0.000484
503970,0.011220,0.045311,0.001486,0.000284,0.039588,0.000747,0.000000,0.000949,0.000000,0.009012,...,0.006779,0.000810,0.000752,0.002054,0.000000,0.002570,0.014169,0.000079,0.001089,0.008937


In [68]:
# Output of some AuthorIds soley testing purposes
V.columns.unique()

Index([      1533,       1535,       1792,       2148,       2310,       2312,
             2586,       2695,       3205,       3288,
       ...
       2001928915, 2002048922, 2002084712, 2002169932, 2002256447, 2002273175,
       2002375343, 2002404048, 2002448241, 2002754832],
      dtype='int64', name='AuthorId', length=7418)

In [75]:
# Create list with top 10 recommendations according to values of dot product
top_indices = V[1535].nlargest(1000000000000).index.tolist()

print(top_indices)

[50847, 35805, 55768, 46501, 29935, 81211, 47111, 75061, 14681, 28662, 70097, 107868, 62058, 133821, 142464, 37638, 63446, 70165, 18762, 115153, 21203, 8969, 83371, 31020, 87689, 193370, 294371, 33387, 92658, 26632, 148513, 81047, 50124, 519, 129353, 103243, 137037, 20431, 119373, 54134, 41463, 203668, 59070, 25625, 23687, 149952, 118398, 150898, 51780, 167760, 38746, 25632, 61726, 21855, 324890, 41246, 17471, 234919, 81284, 59917, 152136, 279369, 168802, 219707, 110071, 154024, 233926, 55859, 49552, 38125, 29253, 58047, 175079, 204842, 82177, 26785, 135210, 46479, 75751, 43101, 23686, 38584, 51532, 87776, 17051, 45539, 21320, 175230, 16587, 167766, 121064, 52177, 47640, 36688, 284322, 31972, 11040, 131841, 370885, 40821, 200194, 284500, 307616, 38564, 173466, 52058, 40290, 78747, 162765, 61439, 357905, 221237, 65287, 161980, 198003, 33085, 136381, 74760, 26181, 8926, 56818, 63131, 236734, 30135, 145024, 17172, 8935, 77004, 209058, 456890, 285468, 94673, 10661, 18589, 51535, 65131, 270

In [76]:
# Create dataframe "df_recomm" for recommendation & fill with RecipeIds
df_recomm = pd.DataFrame({'RecipeId': top_indices})

print(df_recomm)

      RecipeId
0        50847
1        35805
2        55768
3        46501
4        29935
...        ...
1015    329664
1016      9369
1017    316304
1018    106251
1019    397737

[1020 rows x 1 columns]


---
Uncommend to download for streamlit:

In [77]:
# CSV of recommendations for streamlit
# df_recomm.to_csv("../data/nmf_vegan_veganitalian.csv")

---

In [74]:
# Merge information about recipe names with RecipeIds
df_recomm = df_recomm.merge(df_recipes[['RecipeId', 'Name']], on='RecipeId', how='left')
# Rename new column
df_recomm = df_recomm.rename(columns={'Name': 'RecipeName'})

print(df_recomm)

      RecipeId                                         RecipeName
0        50847                                  Roasted Asparagus
1        35805  Easy Peezy Pizza Dough (Bread Machine Pizza Do...
2        55768                    Baja Black Beans, Corn and Rice
3        46501                         Basic Machine French Bread
4        29935                                   1 Pan Fudge Cake
...        ...                                                ...
1015    329664                             Vegan Oatmeal Pancakes
1016      9369                               Low-Fat Spanish Rice
1017    316304                         Cornbread (Egg-Free/Vegan)
1018    106251  Roasted Cauliflower &amp; 16 Roasted Cloves of...
1019    397737                           Habanero Mango Hot Sauce

[1020 rows x 2 columns]
